<a href="https://colab.research.google.com/github/PedroTorrado/Bacon-s-Oracle-Graph/blob/main/Projeto_DAA_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Classe Vertice, Edge e Graph##

In [3]:
# Class Vertice
class Vertex:
    ''' Estrutura de Vértice para um grafo: encapsula um elemento (vertex_id)
        que é o identificador deste nó.

        O elemento (vertex_id) deve ser hashable:
        - Um objeto hashable é aquele que pode ser utilizado como uma chave num dicionário Python.
        - Isto inclui strings, números, tuplas, etc.
    '''

    def __init__(self, vertex_id):
        '''O vértice será inserido no Grafo usando o método insert_vertex(x) que cria um Vertex'''
        self._vertex_id = vertex_id   # Id do vértice (elemento a inserir no grafo)
        self.in_time = None           # Tempo de entrada no vértice (exercício TP3)
        self.out_time = None          # Tempo de saída do vértice (exercício TP3)
        self.status = None            # Marcação de visitado/não visitado (exercício TP3)

    def __hash__(self):
        '''O valor do elemento é usado como hash para o vértice (o elemento deve ser hashable)'''
        return hash(self._vertex_id)  # devolve o hash do elemento

    def __str__(self):
        '''Devolve a representação do objeto vértice em string.'''
        if self.in_time:
            return'v{0}-in({1})-out({2})'.format(self._vertex_id, self.in_time, self.out_time)
        else:
            return'v{0}'.format(self._vertex_id)

    def __eq__(self, vertex):
        return self._vertex_id == vertex._vertex_id # Deve-se garantir que: se hash(vertex)==hash(self), entao vertex==self

    def __lt__(self, vertex):
        return self._vertex_id < vertex._vertex_id

    def __le__(self, vertex):
        return self._vertex_id <= vertex._vertex_id

    def __gt__(self, vertex):
        return self._vertex_id > vertex._vertex_id

    def __ge__(self, vertex):
        return self._vertex_id >= vertex._vertex_id

    def vertex_id(self):
        ''' Devolve o elemento guardado neste vértice.'''
        return self._vertex_id


# Class Edge
class Edge:
    ''' Estrutura de Aresta para um Grafo: (origem, destino) e peso '''

    def __init__(self, vertex_1, vertex_2, weight):
        self._vertex_1 = vertex_1
        self._vertex_2 = vertex_2
        self._weight = weight

    def __hash__(self):
        # Função que mapeia a aresta a uma posição no dicionário (hash map)
        return hash( (self._vertex_1, self._vertex_2) )

    def __str__(self):
        ''' Devolve a representação do objeto aresta em string: (origem, destino)w=peso '''
        return'e({0},{1})w={2}'.format(self._vertex_1, self._vertex_2, self._weight)

    def __eq__(self, other):
        # define igualdade de duas arestas (deve ser consistente com a função hash)
        return self._vertex_1 == other._vertex_1 and self._vertex_2 == other._vertex_2

    def endpoints(self):
        ''' Devolve a tupla (vertex_1, vertex_2) os vértices adjacentes vertex_1 e vertex_2.'''
        return (self._vertex_1, self._vertex_2)

    def cost(self):
        ''' Devolve o peso associado a este arco.'''
        return self._weight

    def opposite(self, vertex):
        ''' Indica o vértice oposto ao vertex nesta aresta
            (apenas se vertex fizer parte da aresta).'''
        if vertex == self._vertex_1:
            return self._vertex_2
        elif vertex == self._vertex_2:
            return self._vertex_1
        else:
            return None

In [4]:
class Graph:
    '''
    Representação de um grafo usando dicionários encadeados (nested dictionaries).

    Atributos:
    ----------
    adjancencies: Dicionário externo que associa um vértice (Vertex) a um
                  mapa de adjacências (dicionario interno)
    vertices: Dicionário auxiliar que associa o id dos vértices do grafo
              a um objeto Vertex (tabela de símbolos).
    n: Número de vértices no Grafo
    m: Número de arestas no Grafo

    ----------
'''
    def __init__(self):
        '''Construtor: Cria um grafo vazio (dicionário de _adjancencies).'''
        self._adjancencies = {}  # dicionário que associa o par chave-valor: <Vertex v, Mapa de adjacências de v>
        self._vertices = {}      # dicionário que associa o par: <id do vértice, objeto Vertex correspondente>
        self._n = 0              # número de vértices do grafo
        self._m = 0              # número de arestas do grafo

    def __str__(self):
        '''Devolve a representação do grafo em string (toString)'''
        if self._n == 0:
            ret = "DAA-Graph: <empty>\n"
        else:
            ret = "DAA-Graph:\n"
            for vertex in self._adjancencies.keys():
                #ret += "vertex-"
                ret += str(vertex) + ": "
                for edge in self.incident_edges(vertex.vertex_id()):
                    ret += str(edge) + "; "
                ret += "\n"
        return ret

    def order(self):
        '''Ordem de um grafo: a quantidade de vértices no Grafo.'''
        return self._n

    def size(self):
        '''Dimensão de um grafo: a quantidade total de arestas do Grafo.'''
        return self._m

    def has_vertex(self, vertex_id):
        '''Verifica se o vértice de id vertex_id está no grafo.'''
        return vertex_id in self._vertices

    def has_edge(self, u_id, v_id):
        '''Verifica se a aresta (u_id, v_id) existe no grafo.'''
        if not self.has_vertex(u_id) or not self.has_vertex(v_id):
            return False
        else:
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            return vertex_v in self._adjancencies[vertex_u]

    def insert_vertex(self, vertex_id):
        '''Insere um novo vértice com o id vertex_id.'''
        if not self.has_vertex(vertex_id):
            vertex = Vertex(vertex_id)
            self._vertices[vertex_id] = vertex  # insere o novo vértice no dicionario de vertices
            self._adjancencies[vertex] = {}     # inicializa o mapa de adjacências deste vértice a vazio
            self._n +=1

    def insert_edge(self, u_id, v_id, weight=0):
        ''' Cria e insere uma nova aresta entre u_id e v_id com peso weight.
            Se a aresta já existe no grafo, atualiza-se o seu peso.
            Também insere os vértices u_id e v_id, caso não existam.'''
        if not self.has_vertex(u_id):
            self.insert_vertex(u_id) # insere novo vertex e atualiza n
        if not self.has_vertex(v_id):
            self.insert_vertex(v_id) # insere novo vertex e atualiza n
        if not self.has_edge(u_id, v_id):
            self._m +=1           # atualiza m apenas se a aresta ainda não existir no grafo
        else:
          #print(f"Existing edge {u_id} and {v_id}. Will only update weight")
          vertex_u = self._vertices[u_id]
          vertex_v = self._vertices[v_id]
          e = Edge(vertex_u, vertex_v, weight)
          self._adjancencies[vertex_u][vertex_v] = e  # coloca v nas adjacências de u
          self._adjancencies[vertex_v][vertex_u] = e  # e u nas adjacências de v (para facilitar a procura de todas as arestas incidentes num vértice)

    def incident_edges(self, vertex_id):
        '''Devolve um iterável (gerador) com todas as arestas de um vértice com id vertex_id.'''
        vertex = self._vertices[vertex_id]
        for edge in self._adjancencies[vertex].values(): # para todas as arestas incidentes em v:
            yield edge

    def degree(self, vertex_id):
        '''Quantidade de arestas incidentes no vértice v.'''
        if not self.has_vertex(vertex_id):
            return 0
        vertex = self._vertices[vertex_id]
        return len(self._adjancencies[vertex])

    def vertices(self):
        '''Devolve um iterável sobre todos os vértices do Grafo (tipo Vertex)'''
        return self._vertices.values()

    def edges(self):
        '''Devolve um iterável sobre todas as arestas do Grafo (sem arestas duplicadas).'''
        visited_edges = set()
        for vertex in self._adjancencies:
            for edge in self._adjancencies[vertex].values():
                if edge not in visited_edges:
                    visited_edges.add(edge)
                    yield edge

    def remove_vertex(self, vertex_id):
        '''Remove o vértice com id vertex_id. Se o vértice não existir, não faz nada.'''
        if not self.has_vertex(vertex_id):
            return

        vertex = self._vertices[vertex_id]
        del self._vertices[vertex_id]

        for v in self._adjancencies[vertex]:
            del self._adjancencies[v][vertex]

        del self._adjancencies[vertex]

        self._n -= 1

    def remove_edge(self, u_id, v_id):
        '''Remove a aresta entre u_id e v_id. Se a aresta não existir, não faz nada.'''
        if self.has_edge(u_id, v_id):
            vertex_u = self._vertices[u_id]
            vertex_v = self._vertices[v_id]
            del self._adjancencies[vertex_u][vertex_v]
            if vertex_u != vertex_v:  # laços são removidos apenas uma vez
                del self._adjancencies[vertex_v][vertex_u]
            self._m -= 1

    def get_edge(self, u_id, v_id):
      ''' Devolve o objeto aresta (Edge) que liga u_id a v_id.
      Devolve None se não forem adjacentes ou se (um d)os vértices não existirem.'''
      #if u_id not in self._adjancencies or v_id not in self._adjancencies:
      if not self.has_edge(u_id, v_id):
        return None
      else:
        vertex_u = self._vertices[u_id]
        vertex_v = self._vertices[v_id]
        return self._adjancencies[vertex_u][vertex_v]


##Grupo 1.a)##

Resumo

Para o desenvolvimento deste projeto decidimos utilizar grafos não orientados, uma vez que isso é um fator que depende dos atores que queremos alcançar, consideramos tomar em consideração o número de ligações entre dois atores (por diferentes filmes) como forma de ter pesos e utilizar uma lista de adjacência pela sua reduzida ordem de complexidade em obter arestas.
De forma a encontrar-mos sempre o caminho mais curto possível e não apenas um caminho, temos também que nos certificar que utilizamos Breath-First Search (BFS) (aka Pesquisa em largura).

• O que são os vértices e as arestas no seu modelo de grafo. Qual foi o critério para esta escolha?

Por exemplo, a sua escolha facilita a implementação de alguma operação específica? Ou faz
com que as operações fiquem mais *eficientes* (em relação ao tempo e ao espaço em memória)?


Os vértices serão os atores e as arestas os filmes que ligam os mesmos. O critério baseia-se na lógica da informação, nós temos um conjunto de atores e como relação entre eles os filmes então achámos lógico que isso se traduza para o grafo como vértices e arestas respetivamente.

Acima de tudo esta implementação vai ajudar na representação final dos dados e a responder a algumas questões mais tarde que possamos fazer como em que filme participou X ator. Isto também significa que ao procurarmos a relação entre os dois atores já temos definido os filmes que também os liga, sem ser necessário procurar essa informação postriormente.

\

• A sua representação do problema resulta em que tipo de grafo (não orientado, orientado,
pesado, com multiarestas, acíclico, cíclico, bipartido, etc)?

A nossa representação resulta num grafo não orientado, com multi-arestas, conexo.


• Que tipo de modificações teve de realizar na classe Graph fornecida (teve de inserir novos
atributos/métodos e porquê?); ou como implementou a sua classe Graph?

A classe graph em comparação à classe dada na aula da semana 7, apenas foi completo com a informação que não foi fornecida no mesmo de forma a poder ser utilizado de forma correta. Para além disso a classe is_directional() foi removida uma vez que será utilizado um grafo não orientado.

---

##Implementação dos ficheiros de dados##



In [5]:
from google.colab import files
import os

filename = "small_dataset_utf8.txt"

# Check if the file exists
if os.path.isfile("/content/" + filename):
  print(f"File '{filename}' already exists in /content/.")
else:
  print(f"File '{filename}' not found in /content/.")
  # Upload a file
  uploaded = files.upload()

  # Check if a file was uploaded
  if uploaded:
    fileName = list(uploaded.keys())[0]  # Get the first filename

    # Read the first 10 lines of the file
    try:
      with open(fileName, 'r') as f:
        for i in range(10):
          line = f.readline()
          print(line, end='')  # Print the line without a newline
        print()  # Add a newline after printing 10 lines
    except FileNotFoundError:
      print(f"Error: File '{fileName}' not found.")


File 'small_dataset_utf8.txt' already exists in /content/.


Tivemos alguma dificuldade em ter a certeza que tinhamos sempre acesso aos ficheiros necessários quando executávamos o processo e por isso decidimos verificar diretamente na diretoria se esta se encontrava com o ficheiro necessário e se não, pedir ao utilizador para importar o mesmo.

Para isso utilizamos os recursos encontrádos em:

https://www.geeksforgeeks.org/how-to-print-all-files-within-a-directory-using-python/
https://colab.research.google.com/notebooks/io.ipynb#scrollTo=hauvGV4hV-Mh

In [10]:
def create_movie_graph(file_name):
  """
  Creates a movie graph from a file with the following format:

  Movie Title (Year) / Actor 1 / Actor 2 / ...

  Args:
      file_name: The name of the file to read.

  Returns:
      A Graph object representing the movie relationships between actors.
  """
  movie_graph = Graph()

  try:
    with open(file_name, 'r') as f:
      for line in f:
        try:
          # Remove leading/trailing spaces and split using '/'
          split_data = line.strip().split('/')

          if len(split_data) < 2:
            raise ValueError("Unexpected line format")

          movie_title = split_data[0]
          year = split_data[1]
          actors = split_data[2:]  # All remaining elements are actors

          # Add movie as vertex
          movie_graph.insert_vertex(movie_title)

          # Add edges between actors for the same movie
          for actor in actors:
            for other_actor in actors:
              if actor != other_actor:  # Avoid self-loops
                movie_graph.insert_edge(movie_title, other_actor)

        except ValueError as e:
          print(f"Error: Encountered unexpected format in line: {line}")
          print(e)

  except FileNotFoundError:
    print(f"Error: File '{file_name}' not found.")

  return movie_graph

# Example usage
movie_graph = create_movie_graph(filename)  # Replace with your file name


In [11]:
print('Ordem:', movie_graph.order(),'Tamanho:', movie_graph.size())
#print(movie_graph)


Ordem: 125249 Tamanho: 15134184
